In [2]:
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
url = "http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx"
ttbkey = os.environ["ttbkey"]

In [7]:
import requests

In [9]:
isbn_type = "ISBN13"
isbn_str = "9791158392239"

In [10]:
res: requests.Response = requests.post(
        url,
        data={
            "TTBKey": ttbkey,
            "itemIdType": isbn_type,
            "ItemId": isbn_str,
            "Cover": "Big",
            "output": "js"
        },
    )

In [17]:
res.text

'{ "version" : "20070901", "title" : "알라딘 상품정보 - 모던 자바스크립트 Deep Dive", "link" : "https:\\/\\/www.aladin.co.kr\\/shop\\/wproduct.aspx?ItemId=251552545", "pubDate" : "Fri, 04 Jun 2021 12:47:42 GMT", "imageUrl" : "http:\\/\\/www.aladin.co.kr\\/ucl_editor\\/img_secur\\/header\\/2010\\/logo.jpg", "totalResults" : 1, "startIndex" : 1, "itemsPerPage" : 1, "query" : "isbn13=9791158392239", "searchCategoryId" : 0, "searchCategoryName" : "", "item" : [  { "title" : "모던 자바스크립트 Deep Dive - 자바스크립트의 기본 개념과 동작 원리", "link" : "http:\\/\\/www.aladin.co.kr\\/shop\\/wproduct.aspx?ItemId=251552545&amp;copyPaper=1&amp;ttbkey=ttbchltjddnrwkd1525001&amp;start=api", "author" : "이웅모 지음", "pubDate" : "2020-09-24", "description" : "자바스크립트를 둘러싼 기본 개념을 정확하고 구체적으로 설명하고, 자바스크립트 코드의 동작 원리를 집요하게 파헤친다. 작성한 코드가 컴퓨터 내부에서 어떻게 동작할 것인지 예측하고, 명확히 설명할 수 있도록 돕는다.", "creator" : "aladin", "isbn" : "K282633473", "isbn13" : "9791158392239", "itemId" : 251552545, "priceSales" : 40500, "priceStandard" : 45000, "stockStatus" : "", "mi

In [18]:
import json

In [21]:
res.text.encode("utf-8")

b'{ "version" : "20070901", "title" : "\xec\x95\x8c\xeb\x9d\xbc\xeb\x94\x98 \xec\x83\x81\xed\x92\x88\xec\xa0\x95\xeb\xb3\xb4 - \xeb\xaa\xa8\xeb\x8d\x98 \xec\x9e\x90\xeb\xb0\x94\xec\x8a\xa4\xed\x81\xac\xeb\xa6\xbd\xed\x8a\xb8 Deep Dive", "link" : "https:\\/\\/www.aladin.co.kr\\/shop\\/wproduct.aspx?ItemId=251552545", "pubDate" : "Fri, 04 Jun 2021 12:47:42 GMT", "imageUrl" : "http:\\/\\/www.aladin.co.kr\\/ucl_editor\\/img_secur\\/header\\/2010\\/logo.jpg", "totalResults" : 1, "startIndex" : 1, "itemsPerPage" : 1, "query" : "isbn13=9791158392239", "searchCategoryId" : 0, "searchCategoryName" : "", "item" : [  { "title" : "\xeb\xaa\xa8\xeb\x8d\x98 \xec\x9e\x90\xeb\xb0\x94\xec\x8a\xa4\xed\x81\xac\xeb\xa6\xbd\xed\x8a\xb8 Deep Dive - \xec\x9e\x90\xeb\xb0\x94\xec\x8a\xa4\xed\x81\xac\xeb\xa6\xbd\xed\x8a\xb8\xec\x9d\x98 \xea\xb8\xb0\xeb\xb3\xb8 \xea\xb0\x9c\xeb\x85\x90\xea\xb3\xbc \xeb\x8f\x99\xec\x9e\x91 \xec\x9b\x90\xeb\xa6\xac", "link" : "http:\\/\\/www.aladin.co.kr\\/shop\\/wproduct.aspx?Ite

In [20]:
json.loads(res.text.encode("utf-8"))

JSONDecodeError: Invalid control character at: line 1 column 1334 (char 1333)